In [1]:

import os 
import sys

import networkx as nx
import numpy as np
import pandas as pd

sys.path.append("../src/modeval")
from modulescomparison import ModulesComparison
from modulecontainers import Modules 

In [25]:
from modeval import ebcubed

In [2]:
sys.path.insert(0,os.path.abspath("../src/modeval/"))


In [3]:
## Get the known RegulonDB network with modification described in inspect_input_elements.ipynb
known_trn = pd.read_csv("../data/Regulatory/QCd_Network.csv", index_col=0)
known_trn.head()

,regulatorId,regulatorName,RegulatorGeneName,regulatedId,regulatedName,function,confidenceLevel
313,RDBECOLIPDC00358,DsrA,dsrA,RDBECOLIGNC00450,hns,-,S
315,RDBECOLIPDC00358,DsrA,dsrA,RDBECOLIGNC00539,lrp,-,S
316,RDBECOLIPDC00358,DsrA,dsrA,RDBECOLIGNC00804,rbsA,-,S
317,RDBECOLIPDC00358,DsrA,dsrA,RDBECOLIGNC00805,rbsB,-,S
318,RDBECOLIPDC00358,DsrA,dsrA,RDBECOLIGNC00806,rbsC,-,S


In [4]:
## Load the annotation E Coli 
annotation = pd.read_csv("../data/Annotation/gene_info.csv")
annotation.head()

,locus_tag,gene_name,old_locus_tag,start,end,strand,gene_product,COG,uniprot
0,b0001,thrL,NaN,190,255,+,thr operon leader peptide,No COG annotation,P0AD86
1,b0002,thrA,NaN,337,2799,+,fused aspartate kinase/homoserine dehydrogenase 1,Amino acid transport and metabolism,P00561
2,b0003,thrB,NaN,2801,3733,+,homoserine kinase,Nucleotide transport and metabolism,P00547
3,b0004,thrC,NaN,3734,5020,+,threonine synthase,Amino acid transport and metabolism,P00934
4,b0005,yaaX,NaN,5234,5530,+,DUF2502 domain-containing protein YaaX,Function unknown,P75616


In [5]:
# Define the set of all known genes (E)
all_genes = annotation.locus_tag.values 

# Define a set of all known regulators
known_regulators = known_trn['RegulatorGeneName'].unique()

# Map gene names to locus tags for comparable output with GENIE3 ouput
annotation_map = annotation[annotation.gene_name.notna()].set_index("gene_name")["locus_tag"].to_dict()
known_trn['targets_tags'] = known_trn["regulatedName"].map(annotation_map)

In [7]:

# Iterate over regulators and gene targets to load a graph from edges
known_ecoli_trn = nx.Graph() 
for i in known_trn[['RegulatorGeneName', 'targets_tags']].itertuples():
    known_ecoli_trn.add_edges_from([tuple([i[1], i[2]])]) 
    
edges_ecoli_trn = known_ecoli_trn.edges

# Find the neighbors of the regulators to define gene targets 
known_modules = []
for reg in known_regulators:
    known_modules.append(list(known_ecoli_trn.neighbors(reg)))

In [8]:
## Load the results from GENIE3 run w/o regulator input
genie3_res = pd.read_csv("../data/predicted_results/GENIE3_5_ET_Ksqrt_nTrees1000_nRegs0_2024-06-11.csv", index_col=0)
genie3_res.head()

,regulatoryGene,targetGene,weight
1,b0573,b0574,0.041188
2,b0575,b0574,0.039607
3,b0574,b0575,0.039262
4,b2000,b2001,0.038284
5,b1973,b0296,0.035970


In [9]:
# Load the predicted network
ecoli_trn = nx.Graph()
ecoli_trn.add_edges_from([tuple(val) for val in genie3_res[['regulatoryGene', 'targetGene']].head(2000).values])


In [10]:
# Filter the network for connections with known regulators
# Append neighborhood connected genes to the list of genesets
regulator_ids = annotation[annotation['gene_name'].isin(known_regulators)]['locus_tag'].values

modules = []
for reg in regulator_ids:
    if reg in ecoli_trn.nodes:
        module = list(ecoli_trn.neighbors(reg))
        modules.append(module)

# Get the sets 
modules_as_sets = [set(module) for module in modules]
print(f"There are {len(modules_as_sets)} modules with known regulators and connected targets")


There are 12 modules with known regulators and connected targets


In [26]:
path = "/Users/john002/Library/CloudStorage/OneDrive-PNNL/Desktop/module_eval_ecoli/data/Regulatory"
file = os.path.join(path, "regulators.csv")
pd.DataFrame(regulator_ids, columns=['regulator']).to_csv(file, index=False)

In [11]:
# Load genesets into Module containers
# Load module containers into comparison container
# A = genie3 predicted
# B = known modules
# C = All genes

ModulesA = Modules(modules_as_sets)
ModulesB = Modules(known_modules)
Modules_genes = Modules(all_genes)
mod_comp = ModulesComparison(ModulesA, ModulesB, all_genes)

In [37]:
# Check the size of modules (Regulator-gene targets)
print(mod_comp.membershipsA.sum(axis = 0))
print(mod_comp.membershipsB.sum(axis = 0))

M0      5
M1      1
M2      2
M3      4
M4     12
M5      1
M6      1
M7      2
M8      5
M9      2
M10     1
M11     6
dtype: uint64
M0       8
M1      12
M2      22
M3       8
M4       6
        ..
M124    37
M125    11
M126     9
M127     8
M128    34
Length: 129, dtype: uint64


## Comparison of every predicted module associated with a known regulator to known modules

In [19]:
print(f"\
Number of predicted modules: {mod_comp.jaccards.shape[0]}\n\
Number of known modules: {mod_comp.jaccards.shape[1]}\n\
For the first predicted module, the best overlap with known modules is: {np.argmax(mod_comp.jaccards[0])}\n\
With a jaccard index score of: {mod_comp.jaccards[0].max()}")

Number of predicted modules: 12
Number of known modules: 129
For the first predicted module, the best overlap with known modules is: 53
With a jaccard index score of: 0.625


## Best matches for all predicted modules

In [58]:
max_indices_per_row = np.argmax(mod_comp.jaccards, axis=1)
max_jaccard_per_row = np.max(mod_comp.jaccards, axis=1)
for i, (idx_match, score) in enumerate(zip(max_indices_per_row, max_jaccard_per_row)):
    print(f"M{i}, Best match: {idx_match}, Jaccard: {score}")

M0, Best match: 53, Jaccard: 0.625
M1, Best match: 4, Jaccard: 0.16666666666666666
M2, Best match: 118, Jaccard: 0.25
M3, Best match: 107, Jaccard: 0.23529411764705882
M4, Best match: 41, Jaccard: 0.25
M5, Best match: 0, Jaccard: 0.0
M6, Best match: 107, Jaccard: 0.058823529411764705
M7, Best match: 44, Jaccard: 0.07142857142857142
M8, Best match: 111, Jaccard: 0.2631578947368421
M9, Best match: 66, Jaccard: 0.25
M10, Best match: 91, Jaccard: 0.025
M11, Best match: 95, Jaccard: 0.7142857142857143


## Compute precision recall scores

In [22]:
scores = mod_comp.score(
    baselines=None, 
    scorenames=['rp', 'rr']
)

In [85]:
print(f"recoveries: Evaluated on predicted modules \
({scores['recoveries'].shape[0]}), \
non-zero values: {(scores['recoveries']>0).sum()}, \
average:{scores['recoveries'].mean():.2f} \n\
\n\
relevances: Evaluated on all known modules \
({scores['relevances'].shape[0]}), \
non-zero matches: {(scores['relevances']> 0).sum()}, \
average:{scores['relevances'].mean():.2f}\n\n")

recoveries: Evaluated on predicted modules (12), non-zero values: 11, average:0.24 

relevances: Evaluated on all known modules (129), non-zero matches: 47, average:0.05




## Measures of Module Prediction


#### Variable definitions
G represents all genes  
M a set of known modules  
M' a set of observed modules  
M(g) the modules that contain a gene g  
E(g, M) the set of genes that are together with g in at least one module M

### Precision/Recall

Precision is define as:  

$ {\mathrm{Precision}} = \frac{1}{{\left| G \right|}}\mathop {\sum}\limits_{g \in G} {\frac{1}{{\left| {E\left( {g,M\prime } \right)} \right|}}} \mathop {\sum}\limits_{g{\prime} \in E\left( {g,M{\prime}} \right)}{\frac{{\mathrm{min}\left( {\left| {M\prime (g) \cap M\prime (g\prime )\left| , \right|M(g) \cap M(g\prime )} \right|} \right) \cdot {\mathrm{\Phi (}}g,g\prime {\mathrm{)}}}}{{\left| {M\prime (g) \cap M\prime (g\prime )} \right|}}} $

Where

$ {\mathrm{\Phi }}\left( {g,g\prime } \right) = \frac{1}{{\left| {M\prime \left( {g,g\prime } \right)} \right|}}\mathop {\sum}\limits_{m\prime \in M\prime \left( {g,g\prime } \right)} {\mathop {{{\it{\mathrm{max}}}}}\limits_{m \in M\left( {g,g\prime } \right)} \,{\mathrm{Jaccard}}\left( {m\prime ,m} \right)} $

Recall is calculated the same way but with M' and M switched

## Recovery Relevance 
The recovery and relevance score assess whether an observed module can be matched with a known module.  

Relevance can be defined as:

$ {\mathrm{Relevance}} = \frac{1}{{\left| {M\prime } \right|}}\mathop {\sum}\limits_{m\prime \in M\prime } {\mathop {{\mathrm{max}}}\limits_{m \in M} \,{\mathrm{Jaccard}}\left( {m\prime ,m} \right)} $ 

And Recovery is the inverse (M' and M switched)

Based on the recoveries and relevances, it seems that our observed modules map onto the known modules with not a particularly high degree of accuracy, but not a particualrly low degree of accuracy either, whereas the known modules map fairly poorly onto the observed modules. This would indicate that a small but nonetheless statistically significant portion of the observed modules are contained within the known modules, while the known modules are not well predicted by the observed model.

The recall and precision values are fairly low as well, indicating that a low number of genes from the observed modules were actually contained within the known modules and an even lower number of genes from the known modules were contaied within the observed modules. This leads to a low F1rp value, as both recall and precision were quite low. 

With a low F1rp value and fairly low recoveries and relevences, it is no surprise that the overall harmonic mean, f1rprr, is also quite low. This is strong evidence that the observed modules represent a poor predictive model. 